<a href="https://colab.research.google.com/github/tjdux/ML/blob/main/11_tree_ensembles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 정형 데이터 & 비정형 데이터
- 정형 데이터 (structured data)
  - 구조를 이루는 데이터
  - CSV나 데이터베이스, 혹은 엑셀에 저장하기 쉬움
- 비정형 데이터 (unstructured data)
  - 정형 데이터의 반대
  - 데이터베이스나 엑셀로 표현하기 어려운 데이터
  - e.g. 텍스트 데이터, 사진, 디지털 음악, ...
- 앙상블 학습 (ensemble learning)
  - 정형 데이터를 다루는 데 가장 뛰어난 성과를 내는 알고리즘
  - 대부분 결정 트리를 기반으로 만들어져 있음
- 신경망 알고리즘
  - 비정형 데이터에 사용
  - 비정형 데이터는 규칙성을 찾기 어려워 전통적인 머신러닝 방법으로는 모델을 만들기 까다로움

## 랜덤 포레스트 (Random Forest)
- 앙상블 학습의 대표 주자 중 하나
- 안정적인 성능으로 널리 사용
- 결정 트리를 랜덤하게 만들어 결정 트리의 숲을 만듦 ➡️ 각 결정 트리의 예측을 사용해 최종 예측
- 부트스트랩 샘플 사용
  - 부트스트랩 (Bootstrap)
    - 데이터 세트에서 중복을 허용하여 데이터를 샘플링하는 방식
    - 랜덤 포레스트는 각 트리를 훈련하기 위해 우리가 입력한 훈련 데이터에서 랜덤하게 샘플을 추출하여 훈련 데이터를 만듦
    - 이때 한 샘플이 중복되어 추출될 수 있음
    - 기본적으로 부트스트랩 샘플은 훈련 세트의 크기와 같게 만듦
- 각 노드를 분할할 때 전체 특성 중에서 일부 특성을 무작위로 고른 다음 이 중에서 최선의 분할을 찾음
  - 분류 모델인 RandomForestClassifier: 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 선택
  - 회귀 모델인 RandomForestRegressor: 전체 특성을 사용
- 기본적으로 100개의 결정 트리를 훈련
- 분류일 때는 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스가 예측 클래스
- 회귀일 때는 각 트리의 예측을 평균
- ➕ 훈련 세트에 과대적합되는 것을 막아주고 검증 세트와 테스트 세트에서 안정적인 성능
- 종종 기본 매개변수 설정만으로도 아주 좋은 결과
- 결정 트리가 지원하는 중요한 매개변수를 모두 제공

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# 데이터 준비
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

- RandomForestClassifier는 기본적으로 100개의 결정 트리를 사용하므로 n_jobs=-1 설정이 바람직
- cross_validate() 함수의 n_jobs 매개변수도 -1로 지정하여 최대한 병렬적으로 교차 검증 수행
- 매개변수 return_train_score = True: 검증 점수 뿐만 아니라 훈련 세트에 대한 점수도 같이 반환 ➡️ 과대적합을 파악하는데 용이

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)

# 교차 검증 수행
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(f"훈련 세트 점수: {np.mean(scores['train_score'])}")
print(f"검증 세트 점수: {np.mean(scores['test_score'])}")

훈련 세트 점수: 0.9973541965122431
검증 세트 점수: 0.8905151032797809


In [3]:
# 특성 중요도
# 랜덤 포레스트의 특성 중요도: 각 결정 트리의 특성 중요도를 취합
rf.fit(train_input, train_target)
rf.feature_importances_

array([0.23167441, 0.50039841, 0.26792718])

- 랜덤 포레스트는 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련
- ➡️ 하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻음
- 과대적합을 줄이고 일반화 성능을 높이는 데 도움

- 자체적으로 모델을 평가하는 점수를 얻을 수 있음
  - OOB (out of bag) 샘플: 부트스트랩 샘플에 포함되지 않고 남는 샘플
  - OOB 샘플을 사용하여 부트스트랩 샘플로 훈련한 결정 트리를 평가 가능
  - 매개변수 oob_score=True: 각 결정 트리의 OOB 점수의 평균을 출력
  - OOB 점수를 사용하면 교차 검증을 대신할 수 있어서 결과적으로 훈련 세트에 더 많은 샘플을 사용 가능

In [4]:
# oob score
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
rf.oob_score_

0.8934000384837406

## 엑스트라 트리 (Extra Tree)
- 랜덤 포레스트와 매우 비슷하게 동작
- 기본적으로 100개의 결정 트리를 훈련
- 결정 트리가 제공하는 대부분의 매개변수를 지원
- 랜덤 포레스트와의 차이점: 부트스트랩 샘플을 사용하지 않음
  - 결정 트리를 만들 때 전체 훈련 세트를 사용
  - 대신 노드를 분할할 때 가장 좋은 분할을 찾는 것이 아닌 무작위로 분할
  - splitter 매개변수가 'random'인 결정 트리를 사용
- ➖ 낮아진 성능
- ➕ 많은 트리를 앙상블 하기 때문에 과대적합을 막고 검증 세트의 점수를 높이는 효과
- ➕ 랜덤하게 노드를 분할하기 때문에 빠른 계산 속도
- 회귀 버전: ExtraTreesRegressor

In [5]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(f"훈련 세트 점수: {np.mean(scores['train_score'])}")
print(f"검증 세트 점수: {np.mean(scores['test_score'])}")

훈련 세트 점수: 0.9974503966084433
검증 세트 점수: 0.8887848893166506


In [6]:
# 특성 중요도
et.fit(train_input, train_target)
et.feature_importances_

array([0.20183568, 0.52242907, 0.27573525])

## 그레이디언트 부스팅 (Gradient Boosting)
- 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블
- 기본적으로 깊이가 3인 100개의 결정 트리 사용
- ➕ 과대적합에 강하고 일반적으로 높은 일반화 성능 기대
- 경사 하강법을 사용하여 트리를 앙상블에 추가
  - 분류: 로지스틱 손실 함수
  - 회귀: 평균 제곱 오차 함수
- 손실 함수를 조금씩 최소화하는 방향으로 이동 ⬅️ 깊이가 얕은 트리를 사용하는 이유
- 학습률 매개변수로 속도(복잡도) 조절
  - 학습률 매개변수가 크면 복잡하고 훈련 세트에 과대적합된 모델

In [7]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(f"훈련 세트 점수: {np.mean(scores['train_score'])}")
print(f"검증 세트 점수: {np.mean(scores['test_score'])}")

훈련 세트 점수: 0.8881086892152563
검증 세트 점수: 0.8720430147331015


- 거의 과대적합이 되지 않음
- 결정 트리의 개수를 늘려도 과대적합에 매우 강함
- 학습률을 증가시키고 트리의 개수를 늘리면 조금 더 성능이 향상될 수 있음

In [8]:
# 트리의 개수 500개, 학습률 0.1에서 0.2로 변경

gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(f"훈련 세트 점수: {np.mean(scores['train_score'])}")
print(f"검증 세트 점수: {np.mean(scores['test_score'])}")

훈련 세트 점수: 0.9464595437171814
검증 세트 점수: 0.8780082549788999


In [9]:
# 특성 중요도
gb.fit(train_input, train_target)
gb.feature_importances_

array([0.15887763, 0.6799705 , 0.16115187])

- ➕ 일반적으로 랜덤 포레스트보다 조금 더 높은 성능
- ➖ 순서대로 트리를 추가하기 때문에 느린 훈련 속도
  - ➡️ n_jobs 매개변수가 없음
- 그레이디언트 부스팅의 회귀 버전: GrandientBoostingRegressor

## 히스토그램 기반 그레디언트 부스팅 (Histogram-based Gradient Boosting)
- 정형 데이터를 다루는 머신러닝 알고리즘 중 가장 인기가 높은 알고리즘
- 입력 특성을 256개의 구간으로 나눔
  - 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있음
  - 256개의 구간 중 하나를 떼어 놓고 누락된 값을 위해 사용 ➡️ 누락된 특성이 있더라도 전처리 필요 없음
- 기본 매개변수에서 안정적인 성능을 얻을 수 있음
- max_iter 매개변수: 트리의 개수 지정
- 과대적합을 잘 억제하면서 그레디언트 부스팅보다 조금 더 높은 성능을 제공

In [10]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(f"훈련 세트 점수: {np.mean(scores['train_score'])}")
print(f"검증 세트 점수: {np.mean(scores['test_score'])}")

훈련 세트 점수: 0.9321723946453317
검증 세트 점수: 0.8801241948619236


- 특성 중요도 계산
  - permutation_importance() 함수: 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지를 관찰하여 어떤 특성이 중요한지를 계산
  - n_repeats 매개변수: 랜덤하게 섞을 횟수 지정

In [11]:
# 훈련 세트의 특성 중요도
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
result.importances_mean

array([0.08876275, 0.23438522, 0.08027708])

In [12]:
# 테스트 세트의 특성 중요도
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
result.importances_mean

array([0.05969231, 0.20238462, 0.049     ])

In [13]:
# 최종적인 테스트 세트에서의 성능
hgb.score(test_input, test_target)

0.8723076923076923

### XGBoost
- tree_method='hist': 히스토그램 기반 그레이디언트 부스팅

In [14]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method="hist", random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)
print(f"훈련 세트 점수: {np.mean(scores['train_score'])}")
print(f"검증 세트 점수: {np.mean(scores['test_score'])}")

훈련 세트 점수: 0.9558403027491312
검증 세트 점수: 0.8782000074035686


### LightGBM

In [16]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(f"훈련 세트 점수: {np.mean(scores['train_score'])}")
print(f"검증 세트 점수: {np.mean(scores['test_score'])}")

훈련 세트 점수: 0.935828414851749
검증 세트 점수: 0.8801251203079884
